In [ ]:
import os, sys
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.NOTSET)

logger.info("Application started successfully.")


In [ ]:
# load environment variables

storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]
NESSIE_URI = os.environ['NESSIE_URI']
REF = "etl"
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']

In [ ]:
# set pyspark configuration
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("Read_csv_from_azure_blob") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "4g")

conf.set("spark.sql.execution.verbose", "false")


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [ ]:
# create namespace gold
namespace = "GOLD"
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {namespace}")
spark.sql(f"SHOW NAMESPACES IN nessie").show()

In [ ]:
# create etl branch
spark.sql(f"USE REFERENCE main IN nessie")
spark.sql(f"DROP BRANCH IF EXISTS {REF}")
spark.sql(f"CREATE BRANCH {REF} IN nessie FROM main")
spark.sql(f"LIST REFERENCES IN nessie").show()


In [ ]:
import notebooks.lib.gold_fact_sales_sql as sql
import importlib
importlib.reload(sql)

source_list = [
    {
        "table_name": "INTERNET_SALES_HEADER",
        "sql_create_table": sql.sql_create_gold_internet_sales_header,
        "sql_merge": sql.sql_merge_gold_internet_sales_header,
        "enabled": True
    },
    {
        "table_name": "INTERNET_SALES_DETAIL",
        "sql_create_table": sql.sql_create_gold_internet_sales_detail,
        "sql_merge": sql.sql_merge_gold_internet_sales_detail,
        "enabled": True
    }
]

In [ ]:
# change branch to etl
spark.sql(f"USE REFERENCE main IN nessie").show()

for item in [item for item in source_list if item["enabled"]]:
    table_name = item["table_name"].upper()
    # create table
    # logger.info(f'creating table if not exists:{table_name}')
    spark.sql(item["sql_create_table"])

    # merge data into gold
    # logger.info(f'merging into: {table_name}')
    df = spark.sql(item["sql_merge"])
    # logger.info(f'data merged successfully')



In [ ]:
spark.sql(f"MERGE BRANCH {REF} INTO main IN nessie").show()
spark.sql(f"DROP BRANCH {REF} IN nessie").show()

In [ ]:
spark.sql("LIST REFERENCES IN nessie").show()